데이터가 Sparse matrix 일 때 cosine distance 를 이용하면 대부분의 거리가 0.9 ~ 1.0 에 가깝습니다. 그렇기 때문에 bag of words model 을 이용하는 term frequency matrix 에서는 k-means++ 가 큰 의미가 없습니다. 

k-means++ 은 distance distribution 이 한쪽에 치우치지 않았을 때 잘 작동합니다. Similar cut initializer 는 term frequency matrix 와 같은 sparse vector 에서 효율적으로 initial points 를 선택하기 위한 방법입니다. 더 자세한 설명은 아래의 블로그를 참고해주세요.

https://lovit.github.io/nlp/machine%20learning/2018/03/19/kmeans_initializer/

실험에 이용한 데이터는 96,136 개의 단어로 표현된 229,253 개의 문서입니다.

In [1]:
sparse_x_path = 'YOUR_SPARSEMATRIX'
vocab_path = 'YOUR_VOCABULARY'


from scipy.io import mmread

def load_x(path):
    return mmread(path).tocsr()

def load_vocab(path):
    with open(path, encoding='utf-8') as f:
        vocabs = [vocab.strip() for vocab in f]
    return vocabs

x = load_x(sparse_x_path)
idx2vocab = load_vocab(vocab_path)

print(x.shape)
print(len(idx2vocab))

(229253, 96136)
96136


idx2vocab 은 96,136 개의 단어들의 idx 에 해당하는 단어가 저장되어 있습니다. 이는 cluster labeling 에 이용됩니다.

In [2]:
print(idx2vocab[22000:22005])

['RT해주', 'RU', 'RUB', 'RUE', 'RUN']


In [3]:
import time
import sys
sys.path.append('../')
import soyclustering

print(soyclustering.__version__)

n_clusters = 500

0.0.1


## Time comparison

### Initializer: similar cut vs. k-means++

k-means++ 을 이용할 경우 (229253, 96136) 데이터에 대하여 initialization 에 856.01 초를 이용합니다.

similar cut 은 비슷한 점들을 initial points 로 선택하지 않으면서도 0.427 초만에 initialization 이 가능합니다. distance distribution 이 0.9 ~ 1.0 에 치우친 경우에는 k-means++ 은 잘 작동하지 않습니다.

In [4]:
from soyclustering import SphericalKMeans

kmeans = SphericalKMeans(
    n_clusters=n_clusters,
    init='k-means++',
    sparsity=None,
    max_iter=10,
    tol=0.0001,
    verbose = True
)

t = time.time()
labels = kmeans.fit_predict(x)
t = time.time() - t

print('process time = {} seconds'.format('%.3f' % t))

initialization_time=856.014305 sec, sparsity=0.00262
n_iter=1, changed=229212, inertia=141131.483, iter_time=241.647 sec, sparsity=0.143
n_iter=2, changed=83383, inertia=110900.124, iter_time=246.245 sec, sparsity=0.145
n_iter=3, changed=37925, inertia=107023.971, iter_time=246.316 sec, sparsity=0.147
n_iter=4, changed=21599, inertia=105667.147, iter_time=246.254 sec, sparsity=0.147
n_iter=5, changed=14517, inertia=105008.016, iter_time=246.318 sec, sparsity=0.147
n_iter=6, changed=11038, inertia=104625.108, iter_time=246.509 sec, sparsity=0.146
n_iter=7, changed=8283, inertia=104386.610, iter_time=246.211 sec, sparsity=0.146
n_iter=8, changed=6239, inertia=104224.627, iter_time=246.285 sec, sparsity=0.146
n_iter=9, changed=4755, inertia=104106.965, iter_time=246.182 sec, sparsity=0.146
n_iter=10, changed=3750, inertia=104027.013, iter_time=246.130 sec, sparsity=0.146
process time = 3315.646 seconds


In [5]:
from soyclustering import SphericalKMeans

kmeans = SphericalKMeans(
    n_clusters=n_clusters,
    init='similar_cut',
    sparsity=None,
    max_iter=10,
    tol=0.0001,
    verbose = True
)

t = time.time()
labels = kmeans.fit_predict(x)
t = time.time() - t

print('process time = {} seconds'.format('%.3f' % t))

initialization_time=0.427326 sec, sparsity=0.00247
n_iter=1, changed=229150, inertia=142147.787, iter_time=243.769 sec, sparsity=0.138
n_iter=2, changed=84713, inertia=110041.110, iter_time=248.189 sec, sparsity=0.139
n_iter=3, changed=35117, inertia=106560.032, iter_time=248.236 sec, sparsity=0.14
n_iter=4, changed=20554, inertia=105445.839, iter_time=248.159 sec, sparsity=0.14
n_iter=5, changed=13870, inertia=104817.127, iter_time=248.019 sec, sparsity=0.141
n_iter=6, changed=10343, inertia=104422.329, iter_time=248.242 sec, sparsity=0.141
n_iter=7, changed=7762, inertia=104124.777, iter_time=248.153 sec, sparsity=0.141
n_iter=8, changed=5984, inertia=103892.740, iter_time=248.273 sec, sparsity=0.14
n_iter=9, changed=4656, inertia=103769.320, iter_time=248.266 sec, sparsity=0.14
n_iter=10, changed=3951, inertia=103696.616, iter_time=248.317 sec, sparsity=0.14
process time = 2479.483 seconds


### soyclustering vs. scikit-learn

각 데이터와 centroids 와의 거리 계산 후, 새로운 cluster 에 데이터를 할당하는 부분이 scikit-learn 과 다르게 구현되어 있습니다. 현재 (versio 0.0.1)는 scikit-learn 의 k-means 보다 약 1.1 배 정도 더 긴 계산시간이 필요합니다.

scikit-learn 은 Euclidean distance 를 이용합니다. L2 normalized 된 centroids 가 아닌 L1 normalized 이기 때문에 Cosine 과 Euclidean 의 값은 차이가 있습니다.

In [6]:
import sklearn
print(sklearn.__version__)

# scikit-learn KMeans use only Euclidean
from sklearn.preprocessing import normalize
x_ = normalize(x)

0.19.1


In [7]:
from sklearn.cluster import KMeans

kmeans_sklearn = KMeans(
    n_clusters=n_clusters,
    init='k-means++',
    n_init=1,
    max_iter=10,
    tol=0.0001,
    verbose=1
)

t = time.time()
_label = kmeans_sklearn.fit_predict(x_)
t = time.time() - t

print('process time = {} seconds'.format('%.3f' % t))

Initialization complete
Iteration  0, inertia 194448.505
Iteration  1, inertia 164069.005
Iteration  2, inertia 156706.049
Iteration  3, inertia 154387.984
Iteration  4, inertia 153503.695
Iteration  5, inertia 152999.616
Iteration  6, inertia 152637.286
Iteration  7, inertia 152394.202
Iteration  8, inertia 152241.206
Iteration  9, inertia 152116.953
process time = 3171.774 seconds


## cluster labeling

proportion keywords 를 이용하면 cluster centroids 와 vocabulary index 를 이용하여 각 클러스터 별 cluster label 을 찾을 수 있습니다.

각 cluster 별로 weight 가 높은 candidates_topk 개의 후보 중에서 proportion keyword score 가 높은 topk 개의 단어가 cluster labels 로 선택됩니다. 

Proportion keywords 의 원리는 아래의 블로그에 설명되어 있습니다.

https://lovit.github.io/nlp/machine%20learning/2018/03/21/kmeans_cluster_labeling/

In [8]:
from soyclustering import proportion_keywords

keywords = proportion_keywords(
    kmeans.cluster_centers_,
    labels,
    index2word=idx2vocab,
    topk=30,
    candidates_topk=100
)

keywords 는 list of list of tuple 입니다. Tuple 은 (단어, 키워드 점수)로 이뤄져 있습니다.

In [11]:
keywords[452]

[('클라리넷', 0.9978907897447573),
 ('클라리', 0.9977324549240725),
 ('Clar', 0.9951847594676605),
 ('클라', 0.9923553251177157),
 ('베버', 0.9896206164257484),
 ('5중주', 0.9824885539733323),
 ('Cla', 0.9700346493805939),
 ('오보에', 0.9699907070433522),
 ('cla', 0.9664973682174413),
 ('브람스', 0.9645529839462655),
 ('Cl', 0.9578050363659426),
 ('앙상블', 0.9557603444819697),
 ('앙상', 0.9540240738863477),
 ('음색', 0.9479620400444972),
 ('실내악', 0.9460097955195105),
 ('cl', 0.9387206824795734),
 ('협연', 0.9331645495655623),
 ('악기', 0.9245579038570346),
 ('연주자', 0.9236457582935793),
 ('오케스트라', 0.9210190422389893),
 ('졸업', 0.9199377496036282),
 ('피아노를', 0.9161724598749406),
 ('오케', 0.9151523838008557),
 ('for', 0.9106742383854735),
 ('fo', 0.9025031537451301),
 ('독주', 0.8988433703074216),
 ('선율', 0.8971575058931237),
 ('협주곡', 0.8960669135198962),
 ('모차르트', 0.8843961159039025),
 ('위한', 0.883589179149199)]

In [10]:
for cluster_idx, keyword in enumerate(keywords):
    if cluster_idx % 5 == 0:
        print()
    keyword = ' '.join([w for w,_ in keyword])
    print('cluster#{} : {}'.format(cluster_idx, keyword))


cluster#0 : 부산썬렉스 1364 부산썬 0305 8038 슈퍼세 부산썬팅 해운대점 썬렉스 부산유리막코팅 부산광택 반여 부산유 장전 부산열 803 해운대구 장전동 앞유리썬팅 반여동 부산광 해운대 해운 516 더카 금정구 030 136 금정 부산자동차
cluster#1 : 프리미엄중고차 프리미엄중고 프리미엄세 프리미엄 TUR 프리미 프리 TU 디테일링 GDI CVV GD 브릴리언트 디테일 브릴 오피러스 오피 토스카 CV 세차 코팅 LF소나타 LF 매매 제거 2013 쏘나타 깨끗 실내 고객님께
cluster#2 : 터키행진곡 터키 모차르트 행진곡 11번 KV 331 Mozart Moz 행진 178 피아노소나타 론도 177 장조 3악장 파리 피아노를 1악장 피아노 단조 협주곡 Mo 피아 2악장 33 악장 제1 제2 곡이
cluster#3 : 제스트스 에어댐전문 익시온젠쿱 포쿱글라스 제스트k 대구에어댐 스파크2013년 제스트에어 코란도c에 대구에어 제스트 l2 에어댐보 스파크2013 스파크2 k5에어 K3에 제스 frp 2220 포쿱 트폼 라세티프리미어에 zest 아반떼쿠 크루즈에어 스파크에 에어댐 에어댐수 i30에
cluster#4 : 스레 박소 CDX FH ID 2012 2012년형 2012년 파이오니아 조회 소니 페스티벌 LC 공감 이름 블루투스 201 파이 08 아카 12월 07 20 GT 09 06 02 39 04 CD

cluster#5 : Key pdf Sca Song Sonata Sona Son Ke pd So Piano Pia Pi Bee Maj Mozart Moz maj major Cello mov Min Cell No Cel Viol Violin Be Mo Ce
cluster#6 : Tak 손발 유혹의 유혹 It Every Make 유혹하 Ever 거란 보여줘 Hav Bab Mak 아이비 둘이 Eve 비춰 product Don Do prod Ta pre it Wor Ev 섹시 shop 내게
cluster#7 : pds1 pds eg pd 